In [1]:
!pip install accelerate -U
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.8 MB/s eta 0:00:00


In [1]:
!pip install evaluate
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.1 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

amazon_dataset = load_dataset('amazon_reviews_multi')

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1200000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/30000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/30000 [00:00<?, ? examples/s]

Dataset amazon_reviews_multi downloaded and prepared to /root/.cache/huggingface/datasets/amazon_reviews_multi/all_languages/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
amazon_dataset

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 1200000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 30000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 30000
    })
})

In [ ]:
amazon_dataset['train'][20000]

{'review_id': 'de_0876255',
 'product_id': 'product_de_0527308',
 'reviewer_id': 'reviewer_de_0704039',
 'stars': 1,
 'review_body': 'Das Paket kam zu Weihnachten ohne Umverpackung. Die Kinder haben dem Paketboten die Türe aufgemacht. Eine Notiz in den Bemerkungen, dass der Versand ohne Umverpackung erfolgt, wäre sehr hilfreich gewesen. Über den Inhalt kann ich noch nichts sagen, ausgepackt wird es erst am heiligen Abend Schade',
 'review_title': 'Zustellung im bebilderten Originalkarton',
 'language': 'de',
 'product_category': 'musical_instruments'}

In [3]:
dataset_en = []
for el in amazon_dataset['train']:
    if el['language'] == 'en':
        el.pop("review_id")
        el.pop("product_id")
        el.pop("reviewer_id")
        el.pop("review_title")
        el.pop("language")
        el.pop("product_category")
        el["text"] = el.pop("review_body")
        el["label"] = el.pop("stars")
        el["label"] = el["label"] - 1
        dataset_en.append(el)

In [6]:
dataset_en[10]

{'text': 'I am disappointed in this purchase. I bought one of these in another color and in size XL and it fit great. This one I bought in white in XL and it fits like a Medium. I cannot wear this top and am really disappointed about that, It is advertised as XL but seriously this top would only fit someone who would be a size s/m. An XXL would have been a better choice for me because the sizing is incorrect on this product.',
 'label': 0}

In [4]:
amazon_dataset['validation'][10]

{'review_id': 'de_0042606',
 'product_id': 'product_de_0907490',
 'reviewer_id': 'reviewer_de_0922684',
 'stars': 1,
 'review_body': 'Das Material schlägt Falten, so daß nicht ordentlich vakuumiert werden kann. Mit Material anderer Hersteller ist mir das bisher nicht passiert.',
 'review_title': 'Vakuumrollen von HAUSWERK für Lebensmittel.',
 'language': 'de',
 'product_category': 'kitchen'}

In [5]:
dataset_en_val = []
for el in amazon_dataset['validation']:
    if el['language'] == 'en':
        el.pop("review_id")
        el.pop("product_id")
        el.pop("reviewer_id")
        el.pop("review_title")
        el.pop("language")
        el.pop("product_category")
        el["text"] = el.pop("review_body")
        el["label"] = el.pop("stars")
        el["label"] = el["label"] - 1
        dataset_en_val.append(el)

In [ ]:
len(dataset_en)

200000

In [ ]:
len(dataset_en_val)

5000

In [7]:
from datasets import Dataset
import pandas as pd

df = pd.DataFrame(dataset_en)

In [8]:
df_val = pd.DataFrame(dataset_en_val)

In [9]:
dataset_amazon = Dataset.from_dict(df)

In [10]:
dataset_amazon_val = Dataset.from_dict(df_val)

In [ ]:
dataset_amazon[10]

{'text': 'I am disappointed in this purchase. I bought one of these in another color and in size XL and it fit great. This one I bought in white in XL and it fits like a Medium. I cannot wear this top and am really disappointed about that, It is advertised as XL but seriously this top would only fit someone who would be a size s/m. An XXL would have been a better choice for me because the sizing is incorrect on this product.',
 'label': 1}

In [11]:
from transformers import AutoTokenizer


tokenizer_bert = AutoTokenizer.from_pretrained('bert-base-uncased')

In [12]:
def tokenize_function_bert(examples):
    return tokenizer_bert(examples["text"], padding="max_length", truncation=True)


tokenized_datasets_bert = dataset_amazon.map(tokenize_function_bert, batched=True)

Map:   0%|          | 0/200000 [00:00<?, ? examples/s]

In [13]:
tokenized_datasets_bert_val = dataset_amazon_val.map(tokenize_function_bert, batched=True)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [14]:
small_train_dataset_bert = tokenized_datasets_bert.shuffle(seed=42).select(range(1000))
small_eval_dataset_bert = tokenized_datasets_bert_val.shuffle(seed=42).select(range(1000))

In [ ]:
small_train_dataset_bert[10]

{'text': 'The padding inside of the burp cloth itself forms a ball after being washed so they do not flatten out pretty. They are nice patterns but unless you plan to air dry they are totally not worth the price!',
 'label': 1,
 'input_ids': [101,
  1996,
  11687,
  4667,
  2503,
  1997,
  1996,
  20934,
  14536,
  8416,
  2993,
  3596,
  1037,
  3608,
  2044,
  2108,
  8871,
  2061,
  2027,
  2079,
  2025,
  4257,
  6528,
  2041,
  3492,
  1012,
  2027,
  2024,
  3835,
  7060,
  2021,
  4983,
  2017,
  2933,
  2000,
  2250,
  4318,
  2027,
  2024,
  6135,
  2025,
  4276,
  1996,
  3976,
  999,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [15]:
from transformers import AutoModelForSequenceClassification


model_bert = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=5)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [16]:
from transformers import TrainingArguments, Trainer

training_args_bert = TrainingArguments(output_dir="test_trainer_bert", evaluation_strategy="epoch", num_train_epochs=4,
                                      per_device_train_batch_size=8, per_device_eval_batch_size=8)

In [17]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [18]:
trainer_bert = Trainer(
    model=model_bert,
    args=training_args_bert,
    train_dataset=small_train_dataset_bert,
    eval_dataset=small_eval_dataset_bert,
    compute_metrics=compute_metrics,
)

In [19]:
trainer_bert.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.201904,0.483000
2,No log,1.201369,0.487000
3,No log,1.363648,0.495000
4,0.783100,1.466130,0.507000


TrainOutput(global_step=500, training_loss=0.7831106567382813, metrics={'train_runtime': 475.3754, 'train_samples_per_second': 8.414, 'train_steps_per_second': 1.052, 'total_flos': 1052472569856000.0, 'train_loss': 0.7831106567382813, 'epoch': 4.0})